# Benchmarking simulation tests 
Simple integration tests where certain results are expected and will be compared to theory.

In [1]:
BOLTZMAN_CONSTANT = 1.38064e-23 # J K−1

# 1st : theoretical collision frequency for MB distribution
def collision_frequency_th_T(T, m, d, n): # per unit of volume and time
    return 0.5*n*4*np.sqrt(BOLTZMAN_CONSTANT*T/m)*d*d*n

def energy(arr, mass): # arr of velocities - shape n x 3
    return 0.5*mass*np.sum(np.linalg.norm(arr, axis =1)**2)

In [3]:
%matplotlib widget

# system
import lppydsmc as ld

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import os
np.random.seed(1111)

## Defining the system

In [ ]:
# Square System :
dz = 0.001
idx_out_walls = [] # no out walls

segments = 0.001*3*np.array([[0,0,1,0], [0,0,0,1], [1,0,1,1], [0,1,1,1]]) # tube
system = SystemCreator(segments)

offsets = ld.system.get_offsets()
system_shape = ld.system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()
extremal_values = system.get_extremal_values()

## Simulation params

In [ ]:
# Simulation params
iterations = 1000
dt = 1e-5 # in sec, should be a fraction of the mean free time
    
# saving params
saving_period = 100
adding_period = 10

# advection
def f(arr, dt):
    return np.zeros(shape = (arr.shape[0], 3))

args = []
scheme = euler_explicit

## Thermalisation

An array of particle initialized randomly on the square with uniform speed is constructed and immediately given to the container for particles.

In [ ]:
# Grid :
mean_number_per_cell = 100
max_number_per_cell = 10*mean_number_per_cell
resolutions = np.array((3,3), dtype = int)

grid = Grid(resolutions, max_number_per_cell)
volume_cell = dz * system_shape[0]/resolutions[0] * system_shape[1]/resolutions[1]

# Density of the gas
density = 3.2e19 # m-3
n_simu = mean_number_per_cell*np.prod(resolutions) # number of particles in the simulated system
n_real = volume_cell * density * np.prod(resolutions) # number of particles in the real system
mr = n_real/n_simu # macro particules ratio = number of particles in the real system / number of macro part in the simulated system
density_dsmc = density/mr

# Particles
part_type = 'I'
charge, mass, radius = 0, get_mass_part(53, 53, 74), 2e-10

# Array
size_array = mean_number_per_cell*np.prod(resolutions) # 2* # we take exactly the right size

print(f'Number of particles : {size_array}')

min_vel, max_vel = -200, 200
x = np.random.uniform(low = extremal_values['min_x'], high = extremal_values['max_x'], size = size_array)
y = np.random.uniform(low = extremal_values['min_y'], high = extremal_values['max_y'], size = size_array)
vx = np.random.uniform(low = min_vel, high = max_vel, size = size_array)
vy = np.random.uniform(low = min_vel, high = max_vel, size = size_array)
vz = np.random.uniform(low = min_vel, high = max_vel, size = size_array)
arr = np.stack((x,y,vx,vy,vz), axis = 1) 
v_mean = np.mean(np.linalg.norm(arr[:,2:], axis = 1))
arr_save = np.copy(arr)

container = Particle(part_type, charge, mass, radius, size_array)
container.add_multiple(arr)
cross_section = container.get_params()[3]

# Energy in the system :
nrj = energy(arr, mass)
target_temperature = (2*nrj/size_array)/(3*BOLTZMAN_CONSTANT)
print(f'Energy in the system : {nrj} J')
print(f'Should relax toward the temperature : {target_temperature} K')

### Simulation

In [ ]:
# NAME tests
from pathlib import Path

dir_path = Path('results/benchmark/')
name = 'thermalization.h5'

saver = Saver(dir_path, name)


In [ ]:
df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) # bucket for the particles - index of particles is the iteration number

# defining useful arrays and ints 
remains = 0 # fractionnal part of the number of particles to inject (it is then passed to the following time step)
averages = np.full(shape = grid.current.shape, fill_value = mean_number_per_cell) # average number of particles per cell
pmax = 2*v_mean*cross_section*np.ones(averages.shape) # max proba per cell in the simu
remains_per_cell = np.zeros(shape = grid.current.shape, dtype = float) # remains per cell for the particles collisions step

# SIMULATING
print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', ' INJECT ', ' DEL ', ' TRY'))
print('{:-^56}'.format(''))

for it in range(iterations): # tqdm
    n1 = container.get_current()
                   
    # injecting particles - no injection
    # new, remains = inject(in_wall, in_vect, debit, vel_std, radius, dt, remains)
    # container.add_multiple(new)
                   
    n2 = container.get_current()-n1
    
    # PHASE : ADVECTING
        # MOVING PARTICLES
    arr = container.get_particles()
    
    if(it%adding_period == 0):
        df = df.append(pd.DataFrame(data=arr, index=[it]*arr.shape[0], columns = ['x','y','vx','vy','vz']))
    
    advect(arr, f, dt, args, scheme) # advect is inplace
    
        # HANDLING BOUNDARIES 
    count = np.full(fill_value = True, shape = arr.shape[0])
    idxes_out = []
    c = 0
    while(np.sum(count, where = count == True) > 0):
        c+=1
        ct, cp = handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_ = make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)
    
    idxes_out = np.concatenate(idxes_out)
    
    # TODO : make delete multiple better - currently the function creates a new array where as we can do it inplace.
    container.delete_multiple(idxes_out)
    
    arr = container.get_particles()
    
    # PHASE : COLLISIONS
        # UPDATING GRID - HARD RESET
        # TODO : change the way it's done
    grid.reset()
    positions = pos_in_grid(arr[:,:2], resolutions, offsets, system_shape)
    particles = convert_to_grid_datatype(positions, new = positions.shape[0])
    grid.add_multiple(particles)
        
        # DSMC
        # TODO: make parallel
    currents = grid.get_currents()
    averages = (it*averages+currents)/(it+1) # may be it too violent ? 
    
    remains_per_cell, nb_colls, pmax, monitor = handler_particles_collisions([arr], grid.get_grid(), currents, dt, averages, pmax, cross_section, volume_cell, mr, remains_per_cell, monitoring = True)
    # PLOTTING AND SAVING (OPTIONAL)
    if(it%saving_period==0 or it == iterations-1): # saving if last iterations too
        saver.save(it = it, append = {
                        'df' : df,
                        'collisions_per_cell' : nb_colls, # evolution of the number of collisions per cell - size : grid.shape[0] x grid.shape[1] (2D)
                        'total_distance' : float(monitor[0]), # evolution of the sum of the distance accross all cells 
                        'total_proba' : float(monitor[1]), # evolution of the sum of proba accross all cells
                        'pmax_per_cell' : pmax,  # evolution of the sum of pmax - per cell (2D)
                        'total_deleted' : len(idxes_out), # evolution of the number of deleted particles per cell (int)
                        'averages_per_cell' : averages
                  })
        
        # resetting dataframe to not use too much memory
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) 
        print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(it, n1, n2, idxes_out.shape[0], c))
   
saver.close()

### Analysis

In [ ]:
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from src.data import Saver
from src.plotting import analysis

dir_path = Path('results/benchmark/')
name = 'thermalization.h5'

store = pd.HDFStore(dir_path/name)

In [ ]:
collisions_per_cell = store['collisions_per_cell'] 
df = store['df']
pmax_per_cell = store['pmax_per_cell']
total_deleted = store['total_deleted']
total_distance = store['total_distance']
total_proba = store['total_proba']

In [ ]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
frames = unique_index[int(0.5*nb_save):nb_save]
fig, axes = analysis.spatial_hist2d(df, frames, 'vy', x_res = 3, y_res = 3, x_step = 1e-3, y_step = 1e-3);

In [ ]:
#analysis.save_fig(fig, path = 'test.png', title = 'vy ($m.s^{-1}$) distribution for each cell - iterations : [500,1000]', dpi = 400, figsize = (8,6))

In [ ]:

fig, axes = analysis.velocity_distribution(df, frames);

In [ ]:
fig, ax = plt.subplots(2,2, constrained_layout = True, figsize = (15,15));
analysis.state(ax[0,0], df.loc[df.index == 250], c = None)
analysis.hist2d(ax[0,1], df, weights = df['vx']) # TODO: add cmap
analysis.nb_particles_evolution(ax[1,0], df, times = dt*np.arange(0,1000,10))
analysis.hist1d(ax[1,1], df, 'x', bins = 50)

In [ ]:
def variance(vx, vy, vz):
    norms = np.sqrt(vx*vx+vy*vy+vz*vz) # N particles - shape : N
    mean = np.mean(norms)
    return np.mean((norms - mean)**2)

In [ ]:
step = adding_period
var_evo = np.array([variance(df.loc[df.index==k]['vx'], df.loc[df.index==k]['vy'], df.loc[df.index==k]['vz']) for k in range(0,iterations,step)])

In [ ]:
var_evo.shape

In [ ]:
fig, ax = plt.subplots()
ax.plot(var_evo*mass/(3*BOLTZMAN_CONSTANT));

In [ ]:
# We can then try to minimize a given function 
a = np.sqrt(BOLTZMAN_CONSTANT*target_temperature/mass) # mean speed at the end (not the most likely one in theory)
sigma_eq = a*a*(3*np.pi-8)/np.pi
print(f'Variance at equilibrium (MB distribution) : {sigma_eq}')

In [ ]:
from scipy.optimize import least_squares

# loss function (to minimize)
def loss_fn(X, var_arr, init_var, eq_var, times):
    # X = [tau] 
    # forced to use a list though
    tau = X[0]
    return np.sum(np.abs(var_arr-(init_var-eq_var)*np.exp(-times/tau)-eq_var))


In [ ]:
tau_init = 1e-3 
var_arr = var_evo
init_var = var_arr[0]
eq_var = sigma_eq

# times
step = 1
t_init = 0
times = np.array([t_init+k*dt for k in range(0,iterations//adding_period,step)])

In [ ]:
results = least_squares(loss_fn, np.array([tau_init]), bounds = ([1e-3*tau_init],[1e3*tau_init]), args = (var_arr,init_var,eq_var,times)).x
tau = results[0]

print(tau)

In [ ]:
def get_Temp(times):
    return (init_var-eq_var)*np.exp(-times/tau)+eq_var


fig, ax = plt.subplots(figsize=(15,10))
ax.set_title("$\sigma_e$ = {} m/s ; $\\tau$ = {} s".format("{:e}".format(np.sqrt(eq_var)),"{:e}".format(tau)))
plt.xlabel("temps (s)",fontsize=18)
plt.ylabel("variance de la vitesse ($m^2/s^2$)",fontsize=16)

plt.plot(times, get_Temp(times), label = '$\sigma^2(t) = (\sigma^2(0)-\sigma^2_e)exp(-t/\\tau)+\sigma^2_e$')
plt.plot(times,var_arr, label = 'Simulation')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.plot(listTime,Temp_smooth)
plt.legend(loc='best',fontsize=16);


## Number of collisions

In [7]:
dz = 0.001
# Grid :
mean_number_per_cell = 100
max_number_per_cell = 10*mean_number_per_cell
resolutions = np.array((3,3), dtype = int)

grid = ld.data_structures.Grid(np.prod(resolutions), max_number_per_cell)
volume_cell = dz * system_shape[0]/resolutions[0] * system_shape[1]/resolutions[1]

# Density of the gas
density = 3.2e19 # m-3
n_simu = mean_number_per_cell*np.prod(resolutions) # number of particles in the simulated system
n_real = volume_cell * density * np.prod(resolutions) # number of particles in the real system
mr = n_real/n_simu # macro particules ratio = number of particles in the real system / number of macro part in the simulated system
density_dsmc = density/mr

# Particles
part_type = 'I'
charge, mass, radius = 0, get_mass_part(53, 53, 74), 2e-10

# Array
size_array = mean_number_per_cell*np.prod(resolutions) # 2* # we take exactly the right size
print(f'Number of particles : {size_array}')

temperature = 300 # K
loc = 0
vel_std = gaussian(temperature, mass)
x = np.random.uniform(low = extremal_values['min_x'], high = extremal_values['max_x'], size = size_array)
y = np.random.uniform(low = extremal_values['min_y'], high = extremal_values['max_y'], size = size_array)
vx = np.random.normal(loc=0.0, scale=vel_std, size = size_array)
vy = np.random.normal(loc=0.0, scale=vel_std, size = size_array)
vz = np.random.normal(loc=0.0, scale=vel_std, size = size_array)
arr = np.stack((x,y,vx,vy,vz), axis = 1) 
v_mean = np.mean(np.linalg.norm(arr[:,2:], axis = 1))

arr_copy = np.copy(arr)

container = ld.data_structures.Particle(part_type, charge, mass, radius, size_array)
container.add_multiple(arr)
cross_section = container.get_params()[3]

# Energy in the system :
nrj = energy(arr, mass)

print(f'Energy in the system : {nrj} J')
T_exp = (2*nrj/size_array)/(3*BOLTZMAN_CONSTANT)
print(f'Should relax toward the temperature : {T_exp} K')

NameError: name 'system_shape' is not defined

In [ ]:
# Simulation params
iterations = 1000
dt = 1e-5 # in sec, should be a fraction of the mean free time
    
# saving params
saving_period = 10
adding_period = 1

# advection
def f(arr, dt):
    return np.zeros(shape = (arr.shape[0], 3))

args = []
scheme = euler_explicit

In [ ]:
# NAME tests
from pathlib import Path

dir_path = Path('results/benchmark/')
name = 'collisions.h5'

saver = Saver(dir_path, name)


In [ ]:
from time import time

In [ ]:
df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) # bucket for the particles - index of particles is the iteration number

# defining useful arrays and ints 
remains = 0 # fractionnal part of the number of particles to inject (it is then passed to the following time step)
averages = np.full(shape = grid.current.shape, fill_value = mean_number_per_cell) # average number of particles per cell
pmax = 2*v_mean*cross_section*np.ones(averages.shape) # max proba per cell in the simu
remains_per_cell = np.zeros(shape = grid.current.shape, dtype = float) # remains per cell for the particles collisions step

# SIMULATING
print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', ' INJECT ', ' DEL ', ' TRY'))
print('{:-^56}'.format(''))

for it in range(iterations): # tqdm

    n1 = container.get_current()
                   
    # injecting particles - no injection
    # new, remains = inject(in_wall, in_vect, debit, vel_std, radius, dt, remains)
    # container.add_multiple(new)
                   
    n2 = container.get_current()-n1
    
    # PHASE : ADVECTING
        # MOVING PARTICLES
    arr = container.get_particles()
    
    if(it%adding_period == 0):
        df = df.append(pd.DataFrame(data=arr, index=[it]*arr.shape[0], columns = ['x','y','vx','vy','vz']))
    
    advect(arr, f, dt, args, scheme) # advect is inplace
    
        # HANDLING BOUNDARIES 
    count = np.full(fill_value = True, shape = arr.shape[0])
    idxes_out = []
    c = 0
    while(np.sum(count, where = count == True) > 0):
        c+=1
        ct, cp = handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_ = make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)
    
    idxes_out = np.concatenate(idxes_out)
    
    # TODO : make delete multiple better - currently the function creates a new array where as we can do it inplace.
    container.delete_multiple(idxes_out)
    
    arr = container.get_particles()
    
    # PHASE : COLLISIONS
        # UPDATING GRID - HARD RESET
        # TODO : change the way it's done
    grid.reset()
    positions = pos_in_grid(arr[:,:2], resolutions, offsets, system_shape)
    particles = convert_to_grid_datatype(positions, new = positions.shape[0])
    grid.add_multiple(particles)
        
        # DSMC
        # TODO: make parallel
    currents = grid.get_currents()
    averages = (it*averages+currents)/(it+1) # may be it too violent ? 
    
    remains_per_cell, nb_colls, pmax, monitor = handler_particles_collisions([arr], grid.get_grid(), currents, dt, averages, pmax, cross_section, volume_cell, mr, remains_per_cell, monitoring = True)
    # PLOTTING AND SAVING (OPTIONAL)
    if(it%saving_period==0 or it == iterations-1): # saving if last iterations too
        saver.save(it = it, append = {
                        'df' : df,
                        'collisions_per_cell' : nb_colls, # evolution of the number of collisions per cell - size : grid.shape[0] x grid.shape[1] (2D)
                        'total_distance' : float(monitor[0]), # evolution of the sum of the distance accross all cells 
                        'total_proba' : float(monitor[1]), # evolution of the sum of proba accross all cells
                        'pmax_per_cell' : pmax,  # evolution of the sum of pmax - per cell (2D)
                        'total_deleted' : len(idxes_out), # evolution of the number of deleted particles per cell (int)
                        'averages_per_cell' : averages
                 })
        
        # resetting dataframe to not use too much memory
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) 
        print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(it, n1, n2, idxes_out.shape[0], c))

saver.close()

# Analysis

In [ ]:
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from src.data import Saver
from src.plotting import analysis

dir_path = Path('results/benchmark/')
name = 'thermalization.h5'

store = pd.HDFStore(dir_path/name)

In [ ]:
collisions_per_cell = store['collisions_per_cell'] 
nb_collisions_per_cell = collisions_per_cell.groupby(collisions_per_cell.index).sum()
df = store['df']
pmax_per_cell = store['pmax_per_cell']
total_deleted = store['total_deleted']
total_distance = store['total_distance']
total_proba = store['total_proba']

In [ ]:
nu_c = collision_frequency_th_T(T = T_exp, m = mass, d=2*radius, n = density)
simu_time = iterations*dt
volume_system = np.prod(resolutions)*volume_cell

In [ ]:
collisions_nb = np.sum(nb_collisions_per_cell)
print('Total number of collision : {:e}'.format(collisions_nb));
print('Equivalent in reality : {:e}'.format(mr*collisions_nb))
print('Expected number of collisions in reality : {:e}'.format(nu_c*simu_time*volume_system))

In [ ]:
fig, ax = plt.subplots()
# number of collisions - evolution
plt.plot(nb_collisions_per_cell.index, nb_collisions_per_cell);

In [ ]:
collisions_per_cell.size

In [ ]:
print('Number of collision per cell : \n');
nb_cells = collisions_per_cell[0].size # number of cells
nb_save = collisions_per_cell.size
S = []
for k in range(nb_cells):
    s = 0
    for i in range(k, nb_save, nb_cells):
        s += collisions_per_cell.iloc[i]
    S.append(s)
print(S)

In [ ]:
proba = total_proba/nb_collisions_per_cell
dist = total_distance/nb_collisions_per_cell
print('Mean proba : {:e}'.format(np.mean(proba)))
print('Mean distance : {:e} m'.format(np.mean(dist)))

fig, ax = plt.subplots(2)
ax[0].plot(proba.index, proba, label = 'proba')
ax[1].plot(dist.index, dist, label = 'distance');